# Instalando Dependecias

In [3]:
pip install rdflib

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install pyvis

Note: you may need to restart the kernel to use updated packages.


# Importaciones

In [5]:
from rdflib import Graph
from SPARQLWrapper import SPARQLWrapper, JSON, N3
from pprint import pprint
import pandas as pd

In [6]:
import pyvis
from pyvis.network import Network
import networkx as nx
import json

# Desarrollo

## 1. Filtrando los datos de la película

In [32]:
sparql = SPARQLWrapper('https://dbpedia.org/sparql')
def detallePelicula(titulo):
    sparql.setQuery('''
        SELECT ?name ?director ?country ?starring ?abstract
        WHERE { dbr:'''+titulo+''' dbp:name ?name .
                dbr:'''+titulo+''' dbo:director ?director .
                dbr:'''+titulo+''' dbp:country ?country .
                dbr:'''+titulo+''' dbp:starring ?starring .
                dbr:'''+titulo+''' dbo:abstract ?abstract .

                FILTER (lang(?abstract) = 'en')
        }''')
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    
    diccionarios = []
    for res in qres["results"]["bindings"]:
        dic ={
            "name":res["name"]["value"],
            "director":res["director"]["value"].split("/")[-1],
            "country":res["country"]["value"],
            "abstract":res["abstract"]["value"],
            "starring":res["starring"]["value"].split("/")[-1]
        }
        diccionarios.append(dic)
    print(len(diccionarios))
    
    dics_string = json.dumps(diccionarios)  
    a_json = json.loads(dics_string)
    dataframe = pd.DataFrame.from_dict(a_json)
    dataframe.drop_duplicates()
    
    return dataframe

In [33]:
detallePelicula('Toy_Story_3')

12


,name,director,country,abstract,starring
0,Toy Story 3,Lee_Unkrich,United States,Toy Story 3 is a 2010 American computer-animat...,Tom Hanks
1,Toy Story 3,Lee_Unkrich,United States,Toy Story 3 is a 2010 American computer-animat...,
2,Toy Story 3,Lee_Unkrich,United States,Toy Story 3 is a 2010 American computer-animat...,John Morris
3,Toy Story 3,Lee_Unkrich,United States,Toy Story 3 is a 2010 American computer-animat...,Ned Beatty
4,Toy Story 3,Lee_Unkrich,United States,Toy Story 3 is a 2010 American computer-animat...,Tim Allen
5,Toy Story 3,Lee_Unkrich,United States,Toy Story 3 is a 2010 American computer-animat...,Wallace Shawn
6,Toy Story 3,Lee_Unkrich,United States,Toy Story 3 is a 2010 American computer-animat...,Joan Cusack
7,Toy Story 3,Lee_Unkrich,United States,Toy Story 3 is a 2010 American computer-animat...,Michael Keaton
8,Toy Story 3,Lee_Unkrich,United States,Toy Story 3 is a 2010 American computer-animat...,Jodi Benson
9,Toy Story 3,Lee_Unkrich,United States,Toy Story 3 is a 2010 American computer-animat...,Estelle Harris


In [8]:
def grafo(df):
    G = nx.Graph()

    for a in df.itertuples():
        if a.starring != "":
            G.add_node(a.starring, size="20",labelHighlightBold=True)
            G.add_edge(a.starring, a.name)

    
    net = Network(notebook=True,directed=True)

    net.from_nx(G)
    #net.show_buttons()
    net.repulsion(node_distance=150, spring_length=100,central_gravity=0.5)
    return net.show("example.html")

In [9]:
def nuevo(titulo):
    dataframe = detallePelicula(titulo)
    dataframe.reset_index()
    dataframe.drop_duplicates()
    return dataframe

def enriquecer(dataframe,titulo):
    df=dataframe.append(detallePelicula(titulo))
    df.reset_index()
    df.drop_duplicates()
    return df

In [10]:
dataframe = nuevo("Toy_Story_3")
grafo(dataframe)

12


In [35]:
dataframe = enriquecer(dataframe,"Forrest_Gump")
dataframe.head(100)
grafo(dataframe)

6


/tmp/ipykernel_13349/3882681488.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=dataframe.append(detallePelicula(titulo))
